# --- `Day 22`: Reactor Reboot ---

In [1]:
import aocd
import re
import heapq
import operator
from collections import Counter, defaultdict, deque
from itertools import combinations
from functools import reduce, lru_cache

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

def first(iterable, default = None):
    return next(iter(iterable), default)

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s):
    return lmap(int, re.findall(r"-?\d+", s))

def words(s):
    return re.findall(r"[a-zA-Z]+", s)

def list_diff(x):
    return [b - a for a, b in zip(x, x[1:])]

def binary_to_int(lst):
    return int("".join(str(i) for i in lst), 2)

def get_column(lst, index):
    return [x[index] for x in lst]

In [2]:
def parse_line(line): 
    return str(line)
    
def parse_input(input):
    return list(map(parse_line, input.splitlines()))

In [ ]:
final_input = parse_input(aocd.get_data(day=22, year=2021))
print(final_input[:5])

In [87]:
test_input = parse_input('''\
on x=10..12,y=10..12,z=10..12
on x=11..13,y=11..13,z=11..13
off x=9..11,y=9..11,z=9..11
on x=10..10,y=10..10,z=10..10
''')

print(test_input)

['on x=10..12,y=10..12,z=10..12', 'on x=11..13,y=11..13,z=11..13', 'off x=9..11,y=9..11,z=9..11', 'on x=10..10,y=10..10,z=10..10']


### Helpers

## Solution 1

In [44]:
def solve_1(input):
    result = set()
    for line in input:
        data = set()
        x1,x2,y1,y2,z1,z2 = ints(line)
        if x1 > 50 or x1 < -50 or x2 > 50 or x2 < -50:
            continue
        if y1 > 50 or y1 < -50 or y2 > 50 or y2 < -50:
            continue
        if z1 > 50 or z1 < -50 or z2 > 50 or z2 < -50:
            continue
        for x in range(x1, x2 + 1):
            for y in range(y1, y2 + 1):
                for z in range(z1, z2 + 1):
                    data.add((x, y, z))
        if line.startswith("on"):
            result = result.union(data)
        else:
            result = result - data
        print(len(result))
    return len(result)

solve_1(test_input)

27
46
38
39


39

In [ ]:
f"Solution 1: {solve_1(final_input)}"

## Solution 2

In [84]:
def overlaps(a, b):
    aminX,amaxX,aminY,amaxY,aminZ,amaxZ = a
    bminX,bmaxX,bminY,bmaxY,bminZ,bmaxZ = b
    
    if not ((aminX <= bmaxX and amaxX >= bminX) and
         (aminY <= bmaxY and amaxY >= bminY) and
         (aminZ <= bmaxZ and amaxZ >= bminZ)):
        return None
    
    ax = set()
    for i in range(aminX, amaxX + 1):
        ax.add(i)   
    bx = set()
    for i in range(bminX, bmaxX + 1):
        bx.add(i)
    xs = ax.intersection(bx)
    
    ay = set()
    for i in range(aminY, amaxY + 1):
        ay.add(i)   
    by = set()
    for i in range(bminY, bmaxY + 1):
        by.add(i)
    ys = ay.intersection(by)
    
    az = set()
    for i in range(aminZ, amaxZ + 1):
        az.add(i)   
    bz = set()
    for i in range(bminZ, bmaxZ + 1):
        bz.add(i)
    zs = az.intersection(bz)
    
    return min(xs), max(xs), min(ys), max(ys), min(zs), max(zs)

print(overlaps((10,12,10,12,10,12), (11,15,11,15,11,15)))

def countCubes(a):
    minX,maxX,minY,maxY,minZ,maxZ = a
    return (maxX - minX + 1) * (maxY - minY + 1) * (maxZ - minZ + 1)

def subtractCubes(a, b):
    pass

print(countCubes((10,12,10,12,10,12)))

(11, 12, 11, 12, 11, 12)
27


In [93]:
def solve_2(input):
    cubes = []

    count = 0
    for line in input:
        count += 1
        print(count, len(input))
        
        x1,x2,y1,y2,z1,z2 = ints(line)
        if x1 > x2:
            x1,x2 = x2,x1
        if y1 > y2:
            y1,y2 = y2,y1
        if z1 > z2:
            z1,z2 = z2,z1
        current = (x1,x2,y1,y2,z1,z2)
        isOn = line.startswith("on")
        
        nextCubes = []
        for cube in cubes:
            nextCubes.append(cube)
            o = overlaps(current, cube[0])
            if o != None:
                overlapOn = isOn if cube[1] != isOn else not isOn
                nextCubes.append((o, overlapOn))
            
        if isOn:
            nextCubes.append((current, isOn))
        cubes = nextCubes
        
    total = 0
    for cube, on in cubes:
        if on:
            total += countCubes(cube)
        else:
            total -= countCubes(cube)
    return total
    
solve_2(test_input)

1 4
2 4
3 4
4 4


39

In [ ]:
f"Solution 2: {solve_2(final_input)}"